## Import data

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('dados/data.csv')
df.head()

,address,district,area,bedrooms,garage,type,rent,total
0,Rua Herval,Belenzinho,21,1,0,Studio e kitnet,2400,2939
1,Avenida São Miguel,Vila Marieta,15,1,1,Studio e kitnet,1030,1345
2,Rua Oscar Freire,Pinheiros,18,1,0,Apartamento,4000,4661
3,Rua Júlio Sayago,Vila Ré,56,2,2,Casa em condomínio,1750,1954
4,Rua Barata Ribeiro,Bela Vista,19,1,0,Studio e kitnet,4000,4654


## Gradio

In [3]:
import gradio as gr
import joblib
import pandas as pd
import numpy as np

# Carregue o modelo e os pré-processadores
modelo = joblib.load("dados/LogisticRegression_melhor_modelo.pkl")
scaler_std = joblib.load("dados/scaler_std.pkl")
ohe_enc = joblib.load("dados/ohe_enc.pkl")


# Defina as features na ordem correta
num_features = ['area', 'bedrooms', 'garage']
cat_features = ['district', 'type']

def preprocess_input(area, bedrooms, garage, district, type_):
    # Crie DataFrame com as features na ordem correta
    data = pd.DataFrame([{
        'area': area,
        'bedrooms': bedrooms,
        'garage': garage,
        'district': district,
        'type': type_
    }])
    # Numéricas
    X_num = scaler_std.transform(data[num_features])
    # Categóricas
    X_cat = ohe_enc.transform(data[cat_features])
    # Junta tudo
    X_final = np.hstack([X_num, X_cat])
    return X_final

def predict(area, bedrooms, garage, district, type_):
    X_input = preprocess_input(area, bedrooms, garage, district, type_)
    pred = modelo.predict(X_input)[0]
    return f"Nível de preço previsto: {pred}"

inputs = [
    gr.Number(label="Área (m²)",minimum=20, maximum=90, step=1),
    gr.Number(label="Quartos", minimum=1,maximum=5, step=1),
    gr.Number(label="Vagas de garagem",minimum=0,maximum=5, step=1),
    gr.Dropdown(
        choices=[bairros for bairros in df['district'].unique() if pd.notna(bairros)],
        label="Bairro"
    ),
    gr.Dropdown(
        choices=["Studio e kitnet", "Casa em condomínio", "Apartamento", "Casa"],
        label="Tipo de imóvel"
    )
]

iface = gr.Interface(
    fn=predict,
    inputs=inputs,
    outputs="text",
    title="Previsão de Nível de Preço do Aluguel em SP"
)

if __name__ == "__main__":
    iface.launch()

/home/fbotero/Documents/POS/pronto/venv/lib/python3.10/site-packages/sklearn/base.py:442: InconsistentVersionWarning: Trying to unpickle estimator LogisticRegression from version 1.7.1 when using version 1.7.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/fbotero/Documents/POS/pronto/venv/lib/python3.10/site-packages/sklearn/base.py:442: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.7.1 when using version 1.7.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/fbotero/Documents/POS/pronto/venv/lib/python3.10/site-packages/sklearn/base.py:442: InconsistentVersionWarning: Trying to unpickle estimator OneHo

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
